In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
pollen_d=pd.read_csv('count_pollen.csv')
pollen_d1=pd.read_csv('count_pollen.csv')
dpp_d = pd.read_csv('dp_pollen.csv')
simp_d = pd.read_csv('sim_pollen.csv')
pollen=pollen_d.values
pollen_imputed=pollen_d1.values
dpp=dpp_d.values
simp=simp_d.values

In [3]:
nrow = len(pollen)
ncol=len(pollen[0])
nr=len(simp)
print(nrow)
print(ncol)
print(nr)

299
9665
9665


In [4]:
imp=np.empty((nrow,ncol))
for i, j in itertools.product(range(0,nrow), range(0,ncol)):
    imp[i][j]=0

In [5]:
imp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
impute=[]
impute=list()
for i, j in itertools.product(range(0,nrow), range(0,ncol)):
    if dpp[i][j]>0.75:
        zval=pollen[i][j]
        temp=[]
        temp=list()
        for k in range(nr):
            if simp[j][k]>0.9:
                val=pollen[i][k]
                temp.append([i,k,val])
        ##regression
        t=np.array(temp)
        n=len(t)
        if(n==0):
            imp[i][j]=np.nan
        if(1<=n<5):
            X=t[:,1]
            y=t[:,2]
            m=round(np.average(y))
            impute.append([i,j,zval,m])
            imp[i][j]=m
            pollen_imputed[i][j]=m
        if(n>=5):
            X=t[:,1]
            y=t[:,2]
            ##Regression
            
            from sklearn.model_selection import train_test_split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
            X_test=np.append(X_test,j)
            y_test=np.append(y_test,zval)
            
            # Fitting Decision Tree Regression to the dataset
            from sklearn.tree import DecisionTreeRegressor
            regressor = DecisionTreeRegressor(random_state=0)
            regressor.fit(X_train.reshape(-1,1), y_train.reshape(-1,1))
            y_pred = regressor.predict(X_test.reshape(-1,1))
            
            df = pd.DataFrame({'Cell_No':X_test.reshape(-1),'Real_Values':y_test.reshape(-1), 'Imputed_Values':y_pred.reshape(-1)})
            m=df.loc[df['Cell_No'] == j, 'Imputed_Values'].squeeze()
            impute.append([i,j,zval,m])
            imp[i][j]=m
            pollen_imputed[i][j]=m
    else:
        imp[i][j]=np.nan

In [7]:
impute

[[0, 80, 0.0, 6.863612478],
 [0, 81, 0.0, 7],
 [0, 89, 0.0, 8.341053159],
 [0, 110, 0.0, 7],
 [0, 128, 0.0, 0.0],
 [0, 148, 0.0, 0.0],
 [0, 152, 0.0, 7.581189882],
 [0, 192, 0.0, 6],
 [0, 269, 0.0, 6.863612478],
 [0, 270, 0.0, 0],
 [0, 316, 0.0, 7],
 [0, 331, 0.0, 5.371865129],
 [0, 399, 0.0, 10.64908209],
 [0, 440, 0.0, 0.0],
 [0, 674, 0.0, 9.146500084],
 [0, 725, 0.0, 8.341053159],
 [0, 768, 0.0, 0.0],
 [0, 775, 0.0, 9],
 [0, 797, 0.0, 0],
 [0, 969, 0.0, 0.0],
 [0, 996, 0.0, 9.364322921],
 [0, 1096, 0.0, 0.0],
 [0, 1144, 0.0, 0],
 [0, 1152, 0.0, 0.0],
 [0, 1196, 0.0, 8.722486343],
 [0, 1222, 0.0, 0.0],
 [0, 1282, 0.0, 0],
 [0, 1290, 0.0, 6.631331788],
 [0, 1304, 0.0, 0.0],
 [0, 1307, 0.0, 0],
 [0, 1346, 0.0, 0.0],
 [0, 1350, 0.0, 0.0],
 [0, 1383, 0.0, 9.146500084],
 [0, 1394, 0.0, 0],
 [0, 1398, 0.0, 7.783731658],
 [0, 1426, 0.0, 7.581189882],
 [0, 1495, 0.0, 6.631331788],
 [0, 1505, 0.0, 0],
 [0, 1538, 0.0, 0],
 [0, 1606, 0.0, 0.0],
 [0, 1645, 0.0, 0.0],
 [0, 1646, 0.0, 0.0],
 [0, 1

In [8]:
imp

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 8.78455155,        nan,
               nan],
       [       nan,        nan,        nan, ..., 7.59687684,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [9]:
pollen_imputed

array([[2.75972773, 0.        , 0.        , ..., 2.75972773, 6.13488238,
        3.64908294],
       [8.35686008, 8.99528641, 0.        , ..., 6.32985805, 0.93198288,
        6.06226192],
       [5.17353884, 4.38992229, 3.41685056, ..., 7.15553848, 0.6825778 ,
        6.7233656 ],
       ...,
       [0.        , 0.        , 0.        , ..., 8.78455155, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 7.59687684, 6.47447685,
        0.        ],
       [8.15983402, 0.        , 0.        , ..., 1.77717137, 0.57095539,
        5.43426485]])

In [10]:
pollen_imputed[0][80]

6.863612478

In [11]:
pollen[0][80]

0.0

In [13]:
pollen_imputed.shape

(299, 9665)

In [15]:
import csv

with open("pollen_imputed.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(pollen_imputed)